In [69]:
import pandas as pd

import mix as mix
import db_column_name as db

# offset can be 9, 21. 33, 45, 57, 69
cn = db.ColumnName()

target_minT = pd.read_csv('./date/31286_103.csv')
mix.set_index_date(target_minT, cn.date)
target_minT = target_minT.sort_index()

X = pd.read_csv('./date/character_31286.csv')
mix.set_index_date(X, cn.date)
X = X.sort_index()

X = X.drop([cn.point], axis=1)
X = X[[x for x in X.columns if 'avg' in x or x == cn.offset]]

X = X.groupby([X.index.year, 
           X.index.month, 
           X.index.day]).mean()
a = pd.DataFrame(X.index.tolist(), 
                 columns=['year','month','day'])
X.index = pd.to_datetime(a)
X = X.drop([cn.offset], axis=1)

target_minT.index = target_minT.index.round('D')

X[cn.value] = target_minT
X = mix.clean_dataset(X)

X, target_minT = mix.getTarget(X)

print(target_minT.shape)
print(X.shape)

(1810, 1)
(1810, 54)


In [70]:
import numpy as np
pd.set_option('precision', 10)

import scipy.sparse
import pickle
import xgboost as xgb

%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg'

from pylab import rcParams
rcParams['figure.figsize'] = 20,10

split_year = 2017

target_train = mix.year_less_eq(target_minT, split_year)
X_train = mix.year_less_eq(X, split_year)
        
target_test = mix.year_great(target_minT, split_year)
X_test = mix.year_great(X, split_year)

print("Size train {}".format(X_train.shape))
print("Size train {}".format(X_test.shape))

#     df = df.drop(['min slptend', 'min advRh850', 'min rainc', 
#                   'min totalprecip', 'min t2', 'min rainnc'], axis=1)

Size train (1659, 54)
Size train (151, 54)


In [73]:
# target_train[(target_train.index.year == 2016)&(target_train.index.month == 1)].plot()